## Instrucciones

Edita, elabora y ejecuta los comandos y responde a las preguntas en este mismo documento. Pero **¡atención!** debes descargarlo (en formato `.ipynb`) para poder entregarlo a través del aula virtual.

## Problema 1
Un centro de secuenciación te envía algunos ficheros de secuencias cortas, que corresponden a nueve muestras, de tres especies diferentes, identificadas con los códigos *sot*, *cav* y *rei*. Los archivos *fastq* están disponibles en la carpeta `data`, junto con otros ficheros en formatos alternativos. Primero, deberás identificar los ficheros que contienen las secuencias cortas en formato *fastq*. A continuación, escoje entre los tres bloques de código siguientes el que necesitas adaptar para analizar la calidad de las secuencias. Por último, corrígelo si contiene errores y adáptalo para que produzca el análisis de calidad de las secuencias cortas de esos ficheros.

In [ ]:
## Opción 1

suppressMessages(library('ShortRead'))
ArchivosFastq <- dir('data', pattern = '*.at')
Calidades <- lapply(ArchivosFastq, function(x) {
    lecturas <- readFastq('data', pattern = x)
    return(width(lecturas))
})
plot(c(0,140), c(0, 0.04), type='n', xlab='Quality?', ylab='Frequency')
x <- 1
for (i in Calidades) {
    lines(density(i), col=x)
    x <- x + 1
}

In [ ]:
## Opción 2

suppressMessages(library('ShortRead'))
ArrosAlForn <- qa('data', type = 'fastq', pattern = '*.fastq')
report(QualityReport, dest = 'InformeQualitat.html')

In [ ]:
## Opción 3

suppressMessages(library('ShortRead'))
archivos <- paste0(rep(c('cav','sot','rei'), each = 3), rep(1:3, 3), '.fastq')
archivos
calidadMedia <- numeric(length = 9)
longitudMedia <- numeric(length = 9)
for (i in 1:9) {
    datos <- readFastq('data', pattern = archivos[i])
    calidadMedia[i] <- mean(as(quality(datos), 'numeric'))
    longitudMedia[i] <- mean(width(datos))
}
data.frame(archivos, calidadMedia, longitudMedia)

Contesta las preguntas siguientes:

1. ¿Has averiguado qué extensión corresponde al formato *fastq* en la carpeta `data`?
2. ¿Para qué sirve la función `width()`?
2. Haz un breve resumen de las conclusiones más importantes del análisis de calidad indicando si crees que la calidad es suficiente y si la calidad es homogénea entre las muestras.
3. Entre las nueve muestras, ¿encuentras alguna que destaque en algún aspecto del resto?
4. ¿Puedes indicar cuál es la longitud media de las lecturas en cada archivo?
5. ¿Cuál sería la probabilidad de què una "C" en cierta posición de una lectura corta en realidad no fuera una citosina en la molécula de la que procede esa lectura, si su calidad es de "J" (es decir, 41)?

## Problema 2

A continuación puedes ejecutar un ejemplo de consulta de la base de datos *European Nucleotide Archive*. Este ejemplo, además de descargar los resultados de la consulta en un archivo de texto, los lee en R y obtiene algunos estadísticos. Comprueba que funciona y familiarízate con el código antes de responder a las preguntas siguientes y resolver el problema planteado. En la página de [taxonomía del NCBI](https://www.ncbi.nlm.nih.gov/taxonomy) encontrarás los códigos **taxid** de cualquier especie o grupo taxonóico. Por ejemplo:

 |Especie        |    taxid|
 |-------------- | --------|
 |*Homo sapiens* |     9606|
 |*Mus musculus* |    10090|
 |Virus          |    10239|
 |Bacterias      |        2|

In [ ]:
library(curl)
portal   <- 'https://www.ebi.ac.uk/ena/portal/api/'
endpoint <- 'search?'
result   <- 'result=read_run'
query    <- paste0('&query=',
                   URLencode('tax_tree(2) AND first_public>=2021-05-01',
                             reserved=TRUE))
fields   <- '&fields=scientific_name,read_count,base_count,library_source,center_name,country'
limit    <- '&limit=0'

URL <- paste0(portal, endpoint, result, query, fields, limit, sep='', collapse='')
curl_download(URL, destfile = 'descarga1.tsv')
resultados <- read.table('descarga1.tsv', header = TRUE, quote = "\"",
                         sep = '\t', na.string = '', stringsAsFactors = FALSE)
head(resultados)
dim(resultados)
sum(resultados$read_count, na.rm = TRUE)
sum(resultados$base_count, na.rm = TRUE)
resultados$genero <- sapply(resultados$scientific_name, function(x) strsplit(x, ' ')[[1]][1])
head(resultados)
sort(table(resultados$genero), decreasing = TRUE)
resultados$country <- sapply(resultados$country, function(x) strsplit(x, ':')[[1]][1])
sort(table(resultados$country), decreasing = TRUE)
sort(table(resultados[resultados$country == 'Russia', 'scientific_name']), decreasing = TRUE)
sort(table(resultados[resultados$country == 'Ireland', 'center_name']), decreasing = TRUE)

#### Preguntas sobre el análisis de ejemplo anterior

1. ¿Cuál es el número de registros de experimentos de secuenciación con lecturas cortas publicados en ENA durante este mes de mayo?

2. ¿Cuál es el género de Bacterias más secuenciado mediante lecturas cortas en el mes de mayo del presente año?

3. ¿Qué especies de bacterias se han dedicado a secuenciar en Rusia durante el mes de mayo? ¿Y en España?

4. ¿Cuántos centros de investigación españoles diferentes han enviado secuencias cortas de Bacterias a la base de datos ENA durante este mes? ¿E irlandeses? 


####  Problema
Elabora ahora tu propia consulta, en un bloque de código nuevo, para investigar los estudios de secuenciación con lecturas cortas en el ratón (*Mus musculus*, taxid 10090) durante el mes de mayo del presenta año. Redacta un pequeño resumen con tus conclusiones. Deberías intentar incluir información sobre: los tipos de tejidos o líneas celulares más secuenciados, los tipos de estudio (*library_source*), el balance entre ratones macho y hembra secuenciados, así como cualquier otro aspecto que te llame la atención. Si prefieres realizar el análisis en [la web de ENA](https://www.ebi.ac.uk/ena/browser/home), o bien en una hoja de cálculo después de descargar los resultados de la búsqueda, también podrás obtener la nota máxima en este apartado. No deberías demorarte más de 20 minutos.


## Problema 3

Para poder realizar este problema, necesitas haber instalado los programas de blast del NCBI. Para hacerlo, solo tienes que ejecutar el código siguiente. Puede tardar unos segundos.

In [ ]:
system2(command = './preparar_ambiente.sh', wait = TRUE)

Dentro de la carptea `data` tienes un archivo fasta llamado `a.fa`. El objetivo es realizar algunas búsquedas de blast con esta *query*. Puedes basarte en los comandos del bloque siguiente, siempre que puedas corregir los errores antes.

1. Busca la proteína **a** en la base de datos *swissprot* (instalada en la carpeta actual) mediante **blastp**, utilizando un umbral de valor E de 0.001. Anota el número de resultados obtenidos.
3. Conociendo la definición del "valor E" y a la vista de los resultados de la búsqueda, ¿sabrías decir cuántos de los resultados obtenidos esperas que sean casuales (es decir, que igualmente podrían haberse producido al azar)?
4. Realiza ahora una búsqueda con **psiblast**, en la misma base de datos y con la misma proteína de partida. ¿Cuántas iteraciones necesitas para alcanzar la convergencia si usas un umbral de inclusión (`-inclusion_ethresh`) de 0.001?
5. Compara el número de resultados obtenidos con **psiblast** con el número de resultados obtenidos antes con **blastp** y explica a qué se debe la diferencia.

In [42]:
system2(command = 'blastn',
        args = c('-db', 'swissprot',
                 '-query', 'data/b.fa',
                 '-out', 'psi1.txt',
                 '-outfmt', '"7 sseqid ssciname scomname sblastname evalue"',
                 '-evalue', '0.005',
                 '-inclusion_ethresh', '0.002',
                 '-num_iterations', '0'),
       stdout=TRUE)

character(0)